In [ ]:
# strat training 
with tf.Session() as sess:
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op],
                           feed_dict = {x : x_train[start:end], y : y_train[start:end]})
            start += batch_size; end += batch_size
            # compute average loss
            avg_loss += loss / iteration
            
        # validation model
        preds = tf.nn.softmax(logits) # apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y,1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        # train accuracy
        cur_train_acc = accuracy.eval({x: x_train, y: y-train})
        # validation accuracy
        cur_val_acc = accuracy.eval({x : x_val, y: y_val})
        # validation loss
        cur_val_loss = loss_op.eval({x : x_val, y: y_val})
        
        print("epoch : "+ str(epoch) + ", train acc : " + str(cur_train_acc)) 
        
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print("early stopped on " + str(epoch))
                    break
                else:
                    print("overfitting waring : " + str(earlystop_cnt))
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # save th variables to file
            save_path = saver.save(sess, "model/model.ckpt")
        prev_train_acc = cur_train_acc
    

ValueError: Cannot feed value of shape (1000, 28, 28) for Tensor 'Placeholder_6:0', which has shape '(?, 784)'